Preprocessing:

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 
import os,glob,skimage,librosa
import librosa.display

import warnings
warnings.filterwarnings("ignore")    
root="/content/drive/MyDrive/Thesis_Keras/"        
csvPth = '/content/drive/MyDrive/Thesis_Keras/UrbanSound8K/metadata/UrbanSound8K.csv'
examplePath = '/content/drive/MyDrive/Thesis_Keras/UrbanSound8K/audio/fold6/85249-2-0-79.wav'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import librosa
import pandas as pd
import numpy as np
import random
import keras
from scipy import signal
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tqdm._tqdm import trange
from keras.utils import np_utils

def readFromCsv(csvpth):
    # 生成数据列表
    # 读取wav文件函数
    #data = pd.read_csv('metadata/UrbanSound8K.csv')
    data = pd.read_csv(csvpth)
    valid_data = data[['slice_file_name', 'fold', 'classID', 'class']][data['end'] - data['start'] >= 2]
    valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + '/' + valid_data['slice_file_name'].astype(
        'str')
    return valid_data


def splitData(current):
  
  D=[]
  i=1

  for row in tqdm(current.itertuples(),total=current.shape[0]):
        #print(row.path)
        #print(row.classID)
        #print(f"{i} out of {len(current)}")
        X, sample_rate = librosa.load("/content/drive/MyDrive/Thesis_Keras/UrbanSound8K/audio/" + row.path, res_type='kaiser_fast', duration=2.97)


        mels = librosa.feature.melspectrogram(X, sr=sample_rate,n_mels=40,n_fft=256,
                         hop_length=32)

        if mels.shape != (40, 128): 
          continue
        feature = mels
        label = row.classID
        D.append((feature,label))
  dataset = D
  X,y=zip(*dataset)
  
  print("type of X is:",type(X))
  y=np.array(y).astype(np.int64)
  y = np.array(np_utils.to_categorical(y, 10))

  return  X,y


def save_npy(nparr,modelName,featureName,subDataset,fold):
    import os
    dirs = "drive/MyDrive/Thesis_Keras/TenFoldDataset/datasetnpy"+'/'+modelName+'/'+featureName+'/'+fold
    
    #Create a directory to place the dataset npy files.
    if not os.path.exists(dirs):
        os.makedirs(dirs)
        print(f"Created directory:{dirs}")
    
    subDataset+='.npy'
    print(subDataset)
    subsetPth=os.path.join(dirs,subDataset)
    #with open(subsetPth, 'w') as f:
    np.save(subsetPth, nparr)
    print(f"save {subDataset} done")
    print(f"Path:{subsetPth}")

def Ten_fold():
    csvPth = '/content/drive/MyDrive/Thesis_Keras/UrbanSound8K/metadata/UrbanSound8K.csv'
    #df=pd.read_csv(csvPth)
    valid_data=readFromCsv(csvPth)
    #valid_data = df[['slice_file_name', 'fold', 'classID', 'class']][df['end'] - df['start']]
    # valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + '/' + valid_data['slice_file_name'].astype(
    #     'str')
    for i in range(10):#Folder : "fold1" to "fold10"
        print(f"fold:{i+1} out of 10")
        current=valid_data[valid_data['fold'] == i+1]
        X,y = splitData(current)
        print("Dataset split done!")
        print("Saving subsets to .npy files!")
        
        save_npy(X, 'MelConv1', 'mel', 'X',f"fold{i+1}")
        save_npy(y, 'MelConv1', 'mel', 'y',f"fold{i+1}")

In [ ]:
Ten_fold() # generate 10-fold training and test data packed in .npy files. Easier to be used in later experiments.

10-fold training-validation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Using the same extracted dataset, and the same folds.

In [ ]:
from sklearn.metrics import accuracy_score
import tensorflow as tf
import numpy as np
from tqdm import tqdm
def generate_loader(i_val):
    train_X = []
    train_y=[]
    X_test=[]
    y_test=[]
    for i in range(10):
        if i + 1 == i_val:
            X_test = np.load(
                "drive/MyDrive/Thesis_Keras/TenFoldDataset/datasetnpy/MelConv1/mel/"+f"fold{i + 1}"+"/X.npy"
                )
            y_test=np.load(
                "drive/MyDrive/Thesis_Keras/TenFoldDataset/datasetnpy/MelConv1/mel/"+f"fold{i + 1}"+"/y.npy"
            )
        else:
            X_train = np.load(
                "drive/MyDrive/Thesis_Keras/TenFoldDataset/datasetnpy/MelConv1/mel/"+f"fold{i + 1}"+"/X.npy"
            )
            y_train = np.load(
                "drive/MyDrive/Thesis_Keras/TenFoldDataset/datasetnpy/MelConv1/mel/"+f"fold{i + 1}"+"/y.npy"
            )

            for item in X_train:
                train_X.append(item)
            for item in y_train:
                train_y.append(item)

    return np.array(train_X),np.array(train_y),np.array(X_test),np.array(y_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Conv2D, Flatten, Reshape, MaxPooling1D, MaxPooling2D, BatchNormalization, TimeDistributed,AveragePooling1D,AveragePooling2D
from tensorflow.keras.optimizers import Adam


def build_model(input_length):
  model = Sequential()
  model.add(Reshape((128, 40), input_shape=(40,128)))
  model.add(Conv1D(8, kernel_size=3, activation='relu', padding='same'))
  model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
  model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
  model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
  model.add(Flatten())
  model.add(Dropout(0.25))
  model.add(Dense(64, activation='relu',
  activity_regularizer=tf.keras.regularizers.l1(0.00001)))
  model.add(Dropout(0.25))
  model.add(Dense(10, activation='softmax', name='y_pred'))

  return model

def train_model(model,X_train,Y_train,X_test,Y_test,foldNum):
  EPOCHS = 100
  # this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
  BATCH_SIZE = 32
  callbacks = []
  # model architecture
  
  # this controls the learning rate
  opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999)
  #callbacks.append(BatchLoggerCallback(BATCH_SIZE, train_sample_count, epochs=EPOCHS))

  # train the neural network
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  model.fit(X_train,Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2,validation_split=0.1,shuffle=True)
  print(model.summary())
  # Use this flag to disable per-channel quantization for a model.
  # This can reduce RAM usage for convolutional models, but may have
  # an impact on accuracy.
  disable_per_channel_quantization = False
  print("Result of fold:"+f"{foldNum}")
  score = model.evaluate(x=X_test,y=Y_test)

In [ ]:
for i in tqdm(range(10)):
  #from tensorflow.compat.v1.keras import backend as K
  import tensorflow as tf

  import os
  # os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
  # config = tf.compat.v1.ConfigProto()#对session进行参数配置
  # config.allow_soft_placement=True #如果你指定的设备不存在，允许TF自动分配设备
  # config.gpu_options.per_process_gpu_memory_fraction=0.7#分配百分之七十的显存给程序使用，避免内存溢出，可以自己调整
  # config.gpu_options.allow_growth = True#按需分配显存，这个比较重要
        
  # sess = tf.compat.v1.Session(config=config)
  # #tf.compat.v1.keras.backend.set_session(sess)
  # #K.set_session(sess)
  X_train,y_train,X_test,y_test = generate_loader(i+1)
  #print(X_train[1])
  #print(y_train[1])
  input_len=len(X_train[0])
  model = build_model(input_len)
  if not i==0:
    model = build_model(input_len)
  
  train_model(model,X_train,y_train,X_test,y_test,i+1)
  model.save("/content/drive/MyDrive/Thesis_Keras/"+"model/saved/HHCNN"+f"fold{i+1}"+".h5")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/100
190/190 - 11s - loss: 2.3832 - accuracy: 0.2418 - val_loss: 1.9338 - val_accuracy: 0.2868 - 11s/epoch - 59ms/step
Epoch 2/100
190/190 - 1s - loss: 1.7848 - accuracy: 0.3228 - val_loss: 1.9666 - val_accuracy: 0.2942 - 601ms/epoch - 3ms/step
Epoch 3/100
190/190 - 1s - loss: 1.6468 - accuracy: 0.3969 - val_loss: 1.7181 - val_accuracy: 0.3774 - 602ms/epoch - 3ms/step
Epoch 4/100
190/190 - 1s - loss: 1.5547 - accuracy: 0.4440 - val_loss: 1.7047 - val_accuracy: 0.4324 - 598ms/epoch - 3ms/step
Epoch 5/100
190/190 - 1s - loss: 1.4739 - accuracy: 0.4742 - val_loss: 1.6495 - val_accuracy: 0.4264 - 599ms/epoch - 3ms/step
Epoch 6/100
190/190 - 1s - loss: 1.4403 - accuracy: 0.5023 - val_loss: 1.9717 - val_accuracy: 0.4844 - 600ms/epoch - 3ms/step
Epoch 7/100
190/190 - 1s - loss: 1.3742 - accuracy: 0.5245 - val_loss: 2.0342 - val_accuracy: 0.5007 - 604ms/epoch - 3ms/step
Epoch 8/100
190/190 - 1s - loss: 1.3423 - accuracy: 0.5383 - val_loss: 1.8231 - val_accuracy: 0.4190 - 596ms/epoch - 3

 10%|█         | 1/10 [01:41<15:12, 101.36s/it]

Epoch 1/100
190/190 - 1s - loss: 2.3643 - accuracy: 0.2486 - val_loss: 1.7968 - val_accuracy: 0.3323 - 1s/epoch - 7ms/step
Epoch 2/100
190/190 - 1s - loss: 1.7760 - accuracy: 0.3309 - val_loss: 1.7634 - val_accuracy: 0.3353 - 599ms/epoch - 3ms/step
Epoch 3/100
190/190 - 1s - loss: 1.6727 - accuracy: 0.3829 - val_loss: 1.7312 - val_accuracy: 0.3709 - 600ms/epoch - 3ms/step
Epoch 4/100
190/190 - 1s - loss: 1.5951 - accuracy: 0.4108 - val_loss: 1.6961 - val_accuracy: 0.4228 - 606ms/epoch - 3ms/step
Epoch 5/100
190/190 - 1s - loss: 1.5610 - accuracy: 0.4337 - val_loss: 1.8075 - val_accuracy: 0.3783 - 604ms/epoch - 3ms/step
Epoch 6/100
190/190 - 1s - loss: 1.5568 - accuracy: 0.4529 - val_loss: 1.8037 - val_accuracy: 0.3858 - 599ms/epoch - 3ms/step
Epoch 7/100
190/190 - 1s - loss: 1.5349 - accuracy: 0.4615 - val_loss: 1.9216 - val_accuracy: 0.3576 - 600ms/epoch - 3ms/step
Epoch 8/100
190/190 - 1s - loss: 1.4555 - accuracy: 0.4748 - val_loss: 1.9261 - val_accuracy: 0.3680 - 607ms/epoch - 3ms/

 20%|██        | 2/10 [03:04<12:04, 90.62s/it] 

...

 80%|████████  | 8/10 [09:38<02:11, 65.75s/it]

Epoch 1/100
191/191 - 1s - loss: 2.3354 - accuracy: 0.2391 - val_loss: 1.8563 - val_accuracy: 0.2482 - 1s/epoch - 7ms/step
Epoch 2/100
191/191 - 1s - loss: 1.7845 - accuracy: 0.3383 - val_loss: 1.7705 - val_accuracy: 0.3722 - 620ms/epoch - 3ms/step
Epoch 3/100
191/191 - 1s - loss: 1.6783 - accuracy: 0.3831 - val_loss: 1.7433 - val_accuracy: 0.3693 - 612ms/epoch - 3ms/step
Epoch 4/100
191/191 - 1s - loss: 1.6026 - accuracy: 0.4341 - val_loss: 1.6836 - val_accuracy: 0.3840 - 618ms/epoch - 3ms/step
Epoch 5/100
191/191 - 1s - loss: 1.5168 - accuracy: 0.4648 - val_loss: 1.8732 - val_accuracy: 0.4003 - 603ms/epoch - 3ms/step
Epoch 6/100
191/191 - 1s - loss: 1.5066 - accuracy: 0.4658 - val_loss: 1.8123 - val_accuracy: 0.4003 - 615ms/epoch - 3ms/step
Epoch 7/100
191/191 - 1s - loss: 1.4566 - accuracy: 0.4874 - val_loss: 1.8485 - val_accuracy: 0.4032 - 608ms/epoch - 3ms/step
Epoch 8/100
191/191 - 1s - loss: 1.4222 - accuracy: 0.4993 - val_loss: 1.8463 - val_accuracy: 0.4402 - 608ms/epoch - 3ms/

 90%|█████████ | 9/10 [10:40<01:04, 64.79s/it]

Epoch 1/100
191/191 - 1s - loss: 2.3123 - accuracy: 0.2497 - val_loss: 1.8686 - val_accuracy: 0.3131 - 1s/epoch - 7ms/step
Epoch 2/100
191/191 - 1s - loss: 1.7691 - accuracy: 0.3552 - val_loss: 1.6168 - val_accuracy: 0.4225 - 594ms/epoch - 3ms/step
Epoch 3/100
191/191 - 1s - loss: 1.6379 - accuracy: 0.3941 - val_loss: 1.5756 - val_accuracy: 0.4490 - 601ms/epoch - 3ms/step
Epoch 4/100
191/191 - 1s - loss: 1.5679 - accuracy: 0.4310 - val_loss: 1.5725 - val_accuracy: 0.4417 - 595ms/epoch - 3ms/step
Epoch 5/100
191/191 - 1s - loss: 1.5026 - accuracy: 0.4479 - val_loss: 1.6768 - val_accuracy: 0.4727 - 599ms/epoch - 3ms/step
Epoch 6/100
191/191 - 1s - loss: 1.5116 - accuracy: 0.4699 - val_loss: 1.5521 - val_accuracy: 0.4549 - 598ms/epoch - 3ms/step
Epoch 7/100
191/191 - 1s - loss: 1.4388 - accuracy: 0.4845 - val_loss: 1.5130 - val_accuracy: 0.5126 - 624ms/epoch - 3ms/step
Epoch 8/100
191/191 - 1s - loss: 1.4144 - accuracy: 0.4915 - val_loss: 1.5264 - val_accuracy: 0.5465 - 667ms/epoch - 3ms/

100%|██████████| 10/10 [11:43<00:00, 70.32s/it]


 You do not need to run following code. Not 10-fold

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Convolution2D, Activation, Flatten, Reshape, MaxPooling1D, MaxPooling2D, BatchNormalization, TimeDistributed,LeakyReLU,SpatialDropout2D,GlobalAveragePooling2D
# from tensorflow.keras.optimizers import Adam,SGD
# from tensorflow.keras import regularizers
# EPOCHS = 100
# # this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
# BATCH_SIZE = 32
# input_length=len(X_train[0])
# callbacks = []

In [ ]:
# model = Sequential()
# model.add(Reshape((128, 40), input_shape=(40,128)))
# model.add(Conv1D(8, kernel_size=3, activation='relu', padding='same'))
# model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
# model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
# model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
# model.add(Flatten())
# model.add(Dropout(0.25))
# model.add(Dense(64, activation='relu',
# activity_regularizer=tf.keras.regularizers.l1(0.00001)))
# model.add(Dropout(0.25))
# model.add(Dense(10, activation='softmax', name='y_pred'))

# # this controls the learning rate
# opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999)

# # train the neural network
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# print(model.summary())
# model.fit(X_train,Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val,Y_val), verbose=2, callbacks=callbacks)

# # Use this flag to disable per-channel quantization for a model.
# # This can reduce RAM usage for convolutional models, but may have
# # an impact on accuracy.
# disable_per_channel_quantization = False

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 128, 40)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 128, 8)            968       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 64, 8)            0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 64, 16)            400       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 32, 16)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 512)              

In [ ]:
# model.save('/content/drive/MyDrive/Thesis_Keras/HH_CNN/data/saved_models/')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# score = model.evaluate(
#         x=X_test,
#         y=Y_test)

47/47 [==============================] - 0s 3ms/step - loss: 1.0185 - accuracy: 0.6838


In [ ]:
# import pandas as pd
# import numpy as np

# pd.plotting.register_matplotlib_converters()
# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# from sklearn.model_selection import GridSearchCV
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
# from tensorflow.keras.utils import to_categorical 
# import os,glob,skimage,librosa
# import librosa.display

# import warnings
# warnings.filterwarnings("ignore")  

In [ ]:
# import sys, os, random
# import tensorflow as tf
# X = np.load('/content/drive/MyDrive/Thesis_Keras/Mel_conv1/data/datasetnpy/melspectro/x_train.npy')
# Y = np.load('/content/drive/MyDrive/Thesis_Keras/Mel_conv1/data/datasetnpy/melspectro/y_train.npy')

# x_train, x_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=1)
# input_length = x_train[0].shape[0]

In [ ]:
# x_test=np.load('/content/drive/MyDrive/Thesis_Keras/Mel_conv1/data/datasetnpy/melspectro/x_test.npy',allow_pickle=True)
# Y_test = np.load('/content/drive/MyDrive/Thesis_Keras/Mel_conv1/data/datasetnpy/melspectro/y_test.npy')

# X_train = x_train
# X_test = x_test
# X_val = x_val
# print(X_train.shape)
# print(X_test.shape)
# print(X_val.shape)

(5383, 40, 128, 1)
(1496, 40, 128, 1)
(599, 40, 128, 1)
